# Converting SAS to CSV

This is a quick notebook to conver the SAS data files that hold the TEL data to CSV.  Once this occurs, we can merge these data with the Reuter's debt data.

In [7]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from sas7bdat import SAS7BDAT

Let's take a look at the files...

In [8]:
files=!ls ../../debt_data/*.sas7bdat
files

['../../debt_data/co_pums11.sas7bdat',
 '../../debt_data/costat12.sas7bdat',
 '../../debt_data/tel_cl_jm.sas7bdat']

Supposedly we can read them right into pandas DFs.

In [9]:
#Create container for each set
df_list=[]

#For each file...
for file_in in files:
    #...create a sas7bdat object...
    with SAS7BDAT(file_in) as f:
        #...and throw a DF version in df_list
        df_list.append(f.to_data_frame())

[costat12.sas7bdat] column count mismatch


Just what is contained in these guys?

In [10]:
for df in df_list:
    print df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28778 entries, 0 to 28777
Data columns (total 41 columns):
STCOU         28778 non-null object
POP_TH18      28778 non-null float64
POP_OV65      28778 non-null float64
TOT_EMP       28778 non-null float64
MFG_EMP       28778 non-null float64
RETL_EMP      28778 non-null float64
T_PUB_SCH     28778 non-null float64
PUB_SCHL      28778 non-null float64
PVT_SCHL      28778 non-null float64
HSLD_PERS     28778 non-null float64
HSG_UNITS     28778 non-null float64
CH_HS_UNT     28778 non-null float64
PRE_1940      28778 non-null float64
VACANT        28778 non-null float64
MDHOMEVAL     28778 non-null float64
MED_INC       28778 non-null float64
PC_INC        28778 non-null float64
LANDAREA      28778 non-null float64
GEN_REV       28778 non-null float64
IGR_ST        28778 non-null float64
TAX_REV       28778 non-null float64
PT_REV        28778 non-null float64
D_GEN_EXP     28778 non-null float64
PC_GEN_EXP    28778 non-null float64
RES_

From Dan's text, it sounds like we want the first DF (which contains selected variables from COStats and PUMS sets) and the third (which contains TEL-related variables).  Do all of these DFs have year and county variables?

In [11]:
for i,df in enumerate(df_list):
    print i,'|','YEAR' in df.columns
    print i,'|','STCOU' in df.columns

0 | True
0 | True
1 | False
1 | True
2 | True
2 | False


It would appear only the first does.  Can the third be effectively merged on FIPS codes?

In [12]:
print df_list[2][[var for var in df_list[2].columns if 'fips' in var.lower()]+['YEAR']].head()
print df_list[2][[var for var in df_list[2].columns if 'fips' in var.lower()]+['YEAR']].tail()

   FIPSST  FIPSCO  YEAR
0       1       3  1990
1       1       3  2000
2       1       3  2001
3       1       3  2002
4       1       3  2003
     FIPSST  FIPSCO  YEAR
658      56      21  2007
659      56      21  2008
660      56      21  2009
661      56      21  2010
662      56      21  2011


FIPS codes are available in the third set.  How are they formatted in the first?

In [13]:
print df_list[0][['YEAR','STCOU']].head()

   YEAR  STCOU
0  1990  00000
1  1990  01000
2  1990  01001
3  1990  01003
4  1990  01005


We can define a new variable in the third set that captures the zero padded versions of `FIPSST` and `FIPSCO` in one variable called `STCOU`.

In [14]:
#Define consolidated FIPS variable
df_list[2]['STCOU']=df_list[2]['FIPSST'].apply(lambda x: str(int(x)).zfill(2))+\
                    df_list[2]['FIPSCO'].apply(lambda x: str(int(x)).zfill(3))

print df_list[2][[var for var in df_list[2].columns if 'fips' in var.lower()]+['YEAR','STCOU']].head()
print df_list[2][[var for var in df_list[2].columns if 'fips' in var.lower()]+['YEAR','STCOU']].tail()

   FIPSST  FIPSCO  YEAR  STCOU
0       1       3  1990  01003
1       1       3  2000  01003
2       1       3  2001  01003
3       1       3  2002  01003
4       1       3  2003  01003
     FIPSST  FIPSCO  YEAR  STCOU
658      56      21  2007  56021
659      56      21  2008  56021
660      56      21  2009  56021
661      56      21  2010  56021
662      56      21  2011  56021


We want to join the third set to the first.

In [15]:
#Capture sets
first=df_list[0].set_index(['YEAR','STCOU'])
third=df_list[2].set_index(['YEAR','STCOU'])

#Sort indices
first.sortlevel(0,inplace=True)
third.sortlevel(0,inplace=True)

#Join sets together
data=first.join(third,rsuffix='_TEL')

data.head().T

YEAR                      1990                                           
STCOU                    00000     01000     01001        01003     01005
POP_TH18          6.360443e+07   1058788     10098        25533      7464
POP_OV65          3.124183e+07    522989      3372        14879      3726
TOT_EMP           1.393809e+08   2061101     11471        40809     12163
MFG_EMP            1.96942e+07    396248      2350         5586      3698
RETL_EMP           2.28855e+07    321969      2163         8007      1577
T_PUB_SCH          4.07376e+07    728252      6847        17054      5156
PUB_SCHL          3.837969e+07    680875      6567        15507      4848
PVT_SCHL               4187099     57284       604         1761       466
HSLD_PERS                 2.63      2.62      2.88         2.62       2.7
HSG_UNITS         1.022637e+08   1670379     12732        50933     10705
CH_HS_UNT                 15.6      13.8      16.1         53.1      14.8
PRE_1940           1.88325e+07    155557       782         3050      1390
VACANT            1.031627e+07    163589       906        13889      1487
MDHOMEVAL                79100     53700     59200        64200     41400
MED_INC                  30056     23597     28337        25712     19389
PC_INC                   14420     11486     11182        12275      9515
LANDAREA               3536342  50750.23    596.01      1596.53       885
GEN_REV           4.115841e+08   4453189     29504        90613     21415
IGR_ST            1.368688e+08   1499474     13444        30831     12048
TAX_REV           1.582935e+08   1237460      7410        24425      5126
PT_REV            1.165025e+08    437295      1728        11424      1655
D_GEN_EXP         3.920146e+08   4307264     24741       100654     21178
PC_GEN_EXP                1618      1073       744         1080       844
RES_POP           2.487099e+08   4040587     34222        98280     25417
PERS_POVT         3.174286e+07    723614      5324        13947      6244
SS_PERS           3.890197e+07    708711      4775        18985      4590
SS_PMT            2.136416e+07    344812      2255         9624      1966
POVT_PCT              12.76301  17.90864  15.55724     14.19109  24.56624
FIPSST                       0         1         1            1         1
FIPSCO                       0         0         1            3         5
...                        ...       ...       ...          ...       ...
_62Health_Social           NaN       NaN       NaN         5674       NaN
_71Arts_Rec                NaN       NaN       NaN         1004       NaN
_72Hotel_Food              NaN       NaN       NaN         7508       NaN
_81OtherServices           NaN       NaN       NaN         2521       NaN
_99Unclassified            NaN       NaN       NaN           12       NaN
SCHENROL_2000              NaN       NaN       NaN        23681       NaN
PC_SCHENROL                NaN       NaN       NaN    0.1686501       NaN
Y20061                     NaN       NaN       NaN            1       NaN
M_P_TAX06                  NaN       NaN       NaN          396       NaN
M_V_OOH06                  NaN       NaN       NaN       167200       NaN
RATE06                     NaN       NaN       NaN  0.002368421       NaN
M_H_INC06                  NaN       NaN       NaN        49451       NaN
BURDEN06                   NaN       NaN       NaN  0.008007927       NaN
SUMLEV                     NaN       NaN       NaN           50       NaN
M_V_OOH99                  NaN       NaN       NaN       122500       NaN
M_H_INC99                  NaN       NaN       NaN        43965       NaN
M_P_TAX99                  NaN       NaN       NaN          359       NaN
BURDEN99                   NaN       NaN       NaN  0.008165586       NaN
RATE99                     NaN       NaN       NaN  0.002930612       NaN
CV_99BURDEN                NaN       NaN       NaN     17.86031       NaN
N_COUNTIES                 NaN       NaN       NaN           67       NaN
NAME

In [16]:
print len(data.columns)
print sorted(data.columns)

194
[u'ACQ_ValAss', u'ASMT_L', u'ASMT_L2', u'ASMT_L3', u'Area', u'BOTH', u'BURDEN05', u'BURDEN06', u'BURDEN99', u'CB', u'CB_E', u'CB_E2', u'CB_E3', u'CB_E4', u'CB_G', u'CB_G2', u'CB_share', u'CFDISC_L', u'CGEXP_L', u'CH_HS_UNT', u'CLEVY_L', u'CLEVY_L2', u'CLEVY_L3', u'CLEVY_L4', u'CRATE_L', u'CRATE_L2', u'CREVU_L', u'CV_99BURDEN', u'County', u'D_GEN_EXP', u'Def', u'Density', u'Dillon_all', u'EAST', u'EDU_MAND', u'EmptoResPop', u'FFDISC_L', u'FIPSCO', u'FIPSCO_TEL', u'FIPSST', u'FIPSST_TEL', u'GEN_REV', u'GEXP_L', u'GL', u'GP_GEXP', u'GP_LEVY', u'GP_LMT', u'GP_RATE', u'GP_REVU', u'GST', u'HOME_STEAD', u'HOME_STEAD2', u'HOME_STEAD3', u'HSG_UNITS', u'HSLD_PERS', u'IGR', u'IGR_ST', u'IIT', u'LANDAREA', u'LEVY_L', u'LIMITS', u'LIT', u'LST', u'MDHOMEVAL', u'ME', u'MED_INC', u'MFDISC_L', u'MFG_EMP', u'MGEXP_L', u'MGEXP_L2', u'MIDDLE', u'MLEVY_L', u'MLEVY_L2', u'MLEVY_L3', u'MLEVY_L4', u'MRATE_L', u'MRATE_L2', u'MREVU_L', u'M_H_INC05', u'M_H_INC06', u'M_H_INC99', u'M_P_TAX05', u'M_P_TAX06', u'

Ok, let's write this to disk.

In [17]:
data.to_csv('../data/tel_data.csv')

## Data Checks

In [19]:
print third.head().T.to_string()

YEAR                            1990                                                                      
STCOU                          01003                   02020          04001          05007           06001
FIPSST                             1                       2              4              5               6
RATE_L                             1                       0              1              0               1
RATE_L2                            1                       0              0              0               1
MRATE_L                            0                       1              0              0               0
CRATE_L                            0                       0              0              0               0
SRATE_L                            0                       0              0              0               0
SRATE_L2                           0                       0              0              0               0
MRATE_L2                           0 

It would be useful to understand how we are impacting the universe of counties with this join (or future joins).  To aid in this effort, let's generate a coverage set.  The index will be the union of the indices from `first` and `third`.  The variables are just boolean values indicating that the data set in question contains a record for the year and county identified in the index.

In [32]:
#Capture union of indices
u_idx=list(set(first.index.values).union(set(third.index.values)))

#Generate county coverage dict
cty_cov=DataFrame({'covariates':[idx in first.index for idx in u_idx],
                   'tel':[idx in third.index for idx in u_idx]},
                  index=pd.MultiIndex.from_tuples(u_idx,names=['Year','FIPS'])).sortlevel(0)

print 'Number of year-counties represented in the COSTAT/PUMS set:',cty_cov['covariates'].sum()
print 'Number of year-counties represented in the institutional set:',cty_cov['tel'].sum()

#Write to disk
cty_cov.to_csv('../data/cty_coverage.csv')

cty_cov

Number of year-counties represented in the COSTAT/PUMS set: 28778
Number of year-counties represented in the institutional set: 663


covariates    tel
Year FIPS                   
1990 00000       True  False
     01000       True  False
     01001       True  False
     01003       True   True
     01005       True  False
     01007       True  False
     01009       True  False
     01011       True  False
     01013       True  False
     01015       True  False
     01017       True  False
     01019       True  False
     01021       True  False
     01023       True  False
     01025       True  False
     01027       True  False
     01029       True  False
     01031       True  False
     01033       True  False
     01035       True  False
     01037       True  False
     01039       True  False
     01041       True  False
     01043       True  False
     01045       True  False
     01047       True  False
     01049       True  False
     01051       True  False
     01053       True  False
     01055       True  False
...               ...    ...
2011 55131       True  False
     55133       True  False
     55135       True  False
     55137       True  False
     55139       True  False
     55141       True  False
     56000       True  False
     56001       True  False
     56003       True  False
     56005       True  False
     56007       True  False
     56009       True  False
     56011       True  False
     56013       True  False
     56015       True  False
     56017       True  False
     56019       True  False
     56021       True   True
     56023       True  False
     56025       True  False
     56027       True  False
     56029       True  False
     56031       True  False
     56033       True  False
     56035       True  False
     56037       True  False
     56039       True  False
     56041       True  False
     56043       True  False
     56045       True  False

[28982 rows x 2 columns]

In [34]:
third.index

MultiIndex(levels=[[1990.0, 2000.0, 2001.0, 2002.0, 2003.0, 2004.0, 2005.0, 2006.0, 2007.0, 2008.0, 2009.0, 2010.0, 2011.0], [u'01003', u'02020', u'04001', u'05007', u'06001', u'08001', u'09001', u'10001', u'11001', u'12001', u'13015', u'15001', u'16001', u'17001', u'18003', u'19013', u'20045', u'21015', u'22005', u'23001', u'24001', u'25001', u'26005', u'27003', u'28033', u'29019', u'30013', u'31055', u'32003', u'33005', u'34001', u'35001', u'36001', u'37001', u'38015', u'39003', u'40017', u'41003', u'42001', u'44003', u'45003', u'46099', u'47001', u'48005', u'49005', u'50007', u'51003', u'53005', u'54003', u'55009', u'56021']],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1